In [ ]:
import pandas as pd
import SeisBlast as sb
import numpy as np
#import matplotlib.pyplot as plt
seisblast_files = '/home/bmelo/bin/SeisBlast/files'

##Import necessary files: Catalogue, Relocated Catalogue, Stations locations##


In [51]:
james_catalogue = pd.read_csv('/mnt/REPO/QUARRY-BLASTS/CATALOGS/quarry-blast-catalog-2013-2014.csv', sep = r'\s+')
james_bulletins_dir = "/mnt/REPO/QUARRY-BLASTS/BULLETINS/"
stations = pd.read_csv(
    "/home/bmelo/bin/Quarry/James_files/Total_Stations_List.txt", 
    delimiter=' ',
    header = 0,
    names=['sta', 'name','lat', 'lon', 'elevation', 'net', 'end'])
#to do in the future: make one catalogue with both locations, before and after relocation
relocated_blasts = pd.read_csv('/home/bmelo/bin/SeisBlast/files/Eire_blasts_relocated.csv')
#merge with the relocated quarry blasts lists to get the relocated events
relocated_catalogue = pd.merge(james_catalogue, relocated_blasts, left_on='EVENT-ID', right_on='id-ev')
relocated_catalogue = relocated_catalogue.sort_values(by=['lon-quarry', 'lat-quarry']).reset_index(drop=True)
relocated_catalogue.head(5)

,EVENT-ID,DATE,TIME,LAT(deg),LAT-ERR(km),LON(deg),LON-ERR(km),DEPTH(km),RMS(s),AZ-GAP(deg),N_PHASES,MAG(ML),lon_ev,lat_ev,id-ev,lon-quarry,lat-quarry
0,dias2013ufgm,2013-10-15,11:40:26.676,51.98154,1.445,-10.21300,2.064,0.0,0.713,163.6,24,1.30,-10.21300,51.98154,dias2013ufgm,-10.177543,51.952300
1,dias2014gote,2014-04-04,10:48:24.192,51.97051,1.711,-10.19241,1.919,0.0,0.761,141.8,22,1.25,-10.19241,51.97051,dias2014gote,-10.177543,51.952300
2,dias2014sjxg,2014-09-19,13:37:28.010,51.97509,1.767,-10.18196,1.883,0.0,0.552,151.3,21,1.32,-10.18196,51.97509,dias2014sjxg,-10.177543,51.952300
3,dias2014gnas,2014-04-03,12:20:07.582,51.90990,2.610,-10.07899,1.920,0.0,0.443,191.3,17,1.11,-10.07899,51.90990,dias2014gnas,-10.069618,51.892000
4,dias2014iyzq,2014-05-08,12:30:09.045,53.51024,1.814,-10.05044,3.756,0.0,0.732,249.4,18,1.38,-10.05044,53.51024,dias2014iyzq,-10.042922,53.501868


##Organize data so we can have a list of travel time at each station-quarry pair##

In [ ]:
quarry_data = pd.DataFrame()

#first loop over the values of the relocated catalogue
for index, row in relocated_catalogue.head(10).iterrows():
    #qry_id = row['quarry-ID']
    ev_id = row['EVENT-ID']
    #get the picks for the event at the index position
    df, info = sb.list_picks(james_catalogue, james_bulletins_dir, ev_id)
    #calculate the travel time in seconds from the origin time
    origin_time = pd.to_datetime((relocated_catalogue.loc[index, 'DATE'] + ' ' + relocated_catalogue.loc[index, 'TIME']), format='%Y-%m-%d %H:%M:%S.%f')
    df['Ptime(s)'] = (df['time']['P'] - origin_time).dt.total_seconds()
    #new dataframe to store the quarry data for the event
    temp_df = pd.DataFrame({
    'quarry-lat': row['lat-quarry'],
    'quarry-lon': row['lon-quarry'],
    'sta': df['sta'], 
    'Ptime(s)': df['Ptime(s)'],
    'ev_id': ev_id,
    })
    #populate quarry_data
    quarry_data = pd.concat([quarry_data, temp_df], axis=0, ignore_index=True)

quarry_data.head(5)

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id
0,51.9523,-10.177543,IA003,31.984,dias2013ufgm
1,51.9523,-10.177543,IA004,26.468,dias2013ufgm
2,51.9523,-10.177543,IA007,30.338,dias2013ufgm
3,51.9523,-10.177543,IA009,36.326,dias2013ufgm
4,51.9523,-10.177543,IAD23,28.709,dias2013ufgm


In [62]:
#group by quarry ID and station and calculate the median, std and count of the picks
quarry_data_group = quarry_data.dropna().groupby(['quarry-lat','quarry-lon','sta']).agg({'Ptime(s)': list, 'ev_id': list}).reset_index()
quarry_data_group.head(10)

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id
0,51.8281,-9.950260,IAD23,[28.046],[dias2014hiyi]
1,51.8281,-9.950260,IGLA,[28.854],[dias2014wvcq]
2,51.8281,-9.950260,UGA10,[12.78],[dias2014wvcq]
3,51.8281,-9.950260,UGA12,[13.926],[dias2014wvcq]
4,51.8281,-9.950260,UGAL1,[12.899],[dias2014wvcq]
5,51.8281,-9.950260,VAL,"[3.758, 4.594]","[dias2014hiyi, dias2014wvcq]"
6,51.8920,-10.069618,IA006,[21.308],[dias2014gnas]
7,51.8920,-10.069618,IA007,[29.047],[dias2014gnas]
8,51.8920,-10.069618,IAD01,[19.226],[dias2014gnas]
9,51.8920,-10.069618,IAD38,[21.234],[dias2014gnas]


In [63]:
#calculate the median, std and count of the picks
quarry_data_group['median'] = quarry_data_group['Ptime(s)'].apply(lambda x: np.median(x))
quarry_data_group['std'] = quarry_data_group['Ptime(s)'].apply(lambda x: sum((i - sum(x)/len(x))**2 for i in x)/len(x))
quarry_data_group['count'] = quarry_data_group['Ptime(s)'].apply(lambda x: len(x))
quarry_data_group.head(10)

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id,median,std,count
0,51.8281,-9.950260,IAD23,[28.046],[dias2014hiyi],28.046,0.000000,1
1,51.8281,-9.950260,IGLA,[28.854],[dias2014wvcq],28.854,0.000000,1
2,51.8281,-9.950260,UGA10,[12.78],[dias2014wvcq],12.780,0.000000,1
3,51.8281,-9.950260,UGA12,[13.926],[dias2014wvcq],13.926,0.000000,1
4,51.8281,-9.950260,UGAL1,[12.899],[dias2014wvcq],12.899,0.000000,1
5,51.8281,-9.950260,VAL,"[3.758, 4.594]","[dias2014hiyi, dias2014wvcq]",4.176,0.174724,2
6,51.8920,-10.069618,IA006,[21.308],[dias2014gnas],21.308,0.000000,1
7,51.8920,-10.069618,IA007,[29.047],[dias2014gnas],29.047,0.000000,1
8,51.8920,-10.069618,IAD01,[19.226],[dias2014gnas],19.226,0.000000,1
9,51.8920,-10.069618,IAD38,[21.234],[dias2014gnas],21.234,0.000000,1


##select event with travel time closest to the mean##

In [ ]:

#now i need to select the ptime and ev_id that are closest to the median
quarry_data['closest-time'] = quarry_data.apply(lambda row: min(row['Ptime(s)'], key=lambda x:abs(x-row['median'])), axis=1)
quarry_data['closest-ev_id'] = quarry_data.apply(lambda row: row['ev_id'][row['Ptime(s)'].index(row['closest-time'])], axis=1)
#now i need to add information about the origin time and magnitude of the event
quarry_data['origin_time'] = quarry_data['closest-ev_id'].apply(lambda x: relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'DATE'].values[0] + ' ' + relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'TIME'].values[0])
quarry_data['mag'] = quarry_data['closest-ev_id'].apply(lambda x: relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'MAG(ML)'].values[0])
#quarry_data.to_csv(f'{seisblast_files}/uniq_quarry-station_list.csv', index=False)
quarry_data

In [40]:
quarry_data[quarry_data['count'] > 10]

,ID,sta,Ptime(s),median,std,count
433,Q5257903,IA003,"[17.168, 17.02, 17.258, 16.785, 16.418, 16.751...",17.0200,0.280555,11
826,Q5277614,IA010,"[20.076, 20.043, 20.372, 20.543, 20.277, 20.51...",20.4195,0.053867,12
827,Q5277614,IA012,"[6.173, 5.441, 5.731, 6.668, 5.729, 6.151, 5.9...",5.9050,0.182128,12
828,Q5277614,IA013,"[14.106, 13.932, 14.135, 14.21, 14.243, 14.523...",14.1725,0.077643,12
933,Q5287869,IA003,"[8.937, 9.151, 8.936, 8.833, 9.221, 9.136, 8.9...",8.9505,0.025384,14
...,...,...,...,...,...,...
5081,Q5483605,IOMK,"[19.477, 19.792, 19.581, 19.674, 19.469, 19.34...",19.4690,0.022974,11
5276,Q5496596,CLGH,"[2.283, 2.663, 2.17, 2.483, 2.264, 2.534, 2.59...",2.3965,0.043692,12
5290,Q5496596,IDGL,"[16.715, 16.326, 16.516, 16.531, 16.651, 16.41...",16.4820,0.053651,11
5506,Q5506678,CLGH,"[7.579, 7.505, 7.481, 7.409, 7.487, 7.548, 7.6...",7.5845,0.012371,14
